In [11]:
#Importing library and thir function
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
from pydub.effects import normalize
from vscode_audio import Audio
from IPython.display import Audio as ipAudio
import matplotlib.pyplot as plt
import noisereduce as nr
import librosa as lr
import librosa.display as lrd
import numpy as np
import time

In [42]:
RATE = 16000
TOP_DB = 25

In [48]:
def audiosegment_to_librosawav(audiosegment):    
    channel_sounds = audiosegment.split_to_mono()
    samples = [s.get_array_of_samples() for s in channel_sounds]

    fp_arr = np.array(samples).T.astype(np.float32)
    fp_arr /= np.iinfo(samples[0].typecode).max
    fp_arr = fp_arr.reshape(-1)

    return fp_arr


def librosa_to_audiosegment(filename):
    y, sr = lr.load(filename, sr=RATE)
    y, _ = lr.effects.trim(y, top_db=TOP_DB) # trim leading and trailing silence
    y = nr.reduce_noise(y, sr=sr) # noise reduction
    # convert from float to uint16
    y = np.array(y * (1<<15), dtype=np.int16)
    audio_segment = AudioSegment(
        y.tobytes(), 
        frame_rate=sr,
        sample_width=y.dtype.itemsize, 
        channels=1
    )
    return audio_segment

def waveshow(y, sr):
    plt.figure(figsize=(14, 5))
    lrd.waveshow(y, sr=sr)
    plt.show()

In [64]:
audioFile = '../Datasets\\tess\YAF_disgust\YAF_back_disgust.wav'

In [65]:
original_sound = librosa_to_audiosegment(audioFile)
augged_sound = normalize(original_sound, headroom=5.0)
# play(augged_sound)

In [66]:
audio_chunks = split_on_silence(augged_sound, min_silence_len=20, silence_thresh=augged_sound.dBFS-16)

In [67]:
len(audio_chunks)

6

In [68]:
# chunks = [audiosegment_to_librosawav(chunk) for chunk in audio_chunks]

In [71]:
print("Playing original...")
play(original_sound)
time.sleep(.5)
print("Playing modified...")
play(augged_sound)
for i, chunk in enumerate(audio_chunks):
    print("Playing chunk{0}".format(i))
    play(chunk)
    # time.sleep(.5)
    output_file = "./test/chunk{0}.wav".format(i)
    print("Exporting file", output_file)
    chunk.export(output_file, format="wav")

Playing original...
Playing modified...
Playing chunk0
Exporting file ./test/chunk0.wav
Playing chunk1
Exporting file ./test/chunk1.wav
Playing chunk2
Exporting file ./test/chunk2.wav
Playing chunk3
Exporting file ./test/chunk3.wav
Playing chunk4
Exporting file ./test/chunk4.wav
Playing chunk5
Exporting file ./test/chunk5.wav
